# Atención primaria España

In [1]:
import pandas as pd
import os

datos [porta estadístico](https://pestadistico.inteligenciadegestion.mscbs.es/publicoSNS/S/base-de-datos-de-clinicos-de-atencion-primaria-bdcap)

# consulta de datasets

In [2]:
psico_prim = pd.read_excel('../data/at_primaria/detalles_psico.xlsx')

In [3]:
psico_prim.columns = psico_prim.columns.str.replace(" ",'_')

In [4]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Grandes_Grupos,Problema_de_Salud,Casos
102378,2016,04-Baleares,Hombre,?100.000 ? /año,15-34 años,P78 - Neurastenia/surmenage,NaN
123243,2016,16-País Vasco,Hombre,Muy baja,15-34 años,P99 - Otros problemas psicológicos/mentales,NaN
3703,2012,02-Aragón,Mujer,<18.000 ? /año,00-14 años,P08 - Ausencia/pérdida de la satisfación sexual,NaN
8994,2012,07-Castilla y León,Mujer,<18.000 ? /año,35-64 años,P19 - Abuso de drogas,343.0
34844,2013,10-Comunidad Valenciana,Mujer,Muy baja,35-64 años,P85 - Retraso mental,703.0


In [5]:
psico_prim.isna().sum()

Año                       0
CCAA                      0
Sexo                      0
Nivel_Renta               0
Grandes_Grupos            0
Problema_de_Salud         0
Casos                182993
dtype: int64

In [6]:
psico_prim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213840 entries, 0 to 213839
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Año                213840 non-null  int64  
 1   CCAA               213840 non-null  object 
 2   Sexo               213840 non-null  object 
 3   Nivel_Renta        213840 non-null  object 
 4   Grandes_Grupos     213840 non-null  object 
 5   Problema_de_Salud  213840 non-null  object 
 6   Casos              30847 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 11.4+ MB


In [7]:
#creación de tabla de correspondencia de CCAA
comunidades_cod = {}
ccaalist = list(psico_prim.CCAA.unique())
for cc in ccaalist:
    comunidades_cod[cc.split("-")[0]] = cc.split("-")[1]
com_cod = pd.DataFrame([[key, comunidades_cod[key]] for key in comunidades_cod.keys()], columns=['Cod', 'CCAA'])
com_cod.to_csv('../data/codigos_com.csv',index=True)

In [8]:
psico_prim.Año.max()

2019

## cambiamos algunos datos categóricos

In [9]:
psico_prim[["CodCA","CCAA"]] = psico_prim.CCAA.str.split("-",expand=True)

In [10]:
list(psico_prim.Nivel_Renta.unique())

['Sin Clasificar',
 '?100.000 ? /año',
 '18.000-99.999 ? /año',
 '<18.000 ? /año',
 'Muy baja']

In [11]:
renta ={'Sin Clasificar':'desconocida',
 '?100.000 ? /año': '+ 100000',
 '18.000-99.999 ? /año': '18000-99999',
 '<18.000 ? /año':'- 18000',
 'Muy baja':'muy_baja'}

In [12]:
psico_prim.Nivel_Renta = psico_prim.Nivel_Renta.replace(renta)

In [13]:
psico_prim.Grandes_Grupos = psico_prim.Grandes_Grupos.str.extract(r'(\d+-*\d+)')
psico_prim.rename({"Grandes_Grupos":"Edad"},axis = 1,inplace = True)
psico_prim.Edad.replace('65','65 +', inplace = True)

In [14]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Edad,Problema_de_Salud,Casos,CodCA
19164,2012,País Vasco,Mujer,18000-99999,15-34,P28 - Incapacidad/minusvalía mental,NaN,16
30221,2013,Castilla la Mancha,Hombre,18000-99999,00-14,P81 - Trastornos hipercinéticos,297.0,08
71134,2015,Andalucía,Hombre,- 18000,65 +,P74 - Trastornos de la ansiedad/estado de ansi...,15161.0,01
202022,2019,Extremadura,Hombre,18000-99999,35-64,P19 - Abuso de drogas,NaN,11
205502,2019,Madrid,Hombre,18000-99999,65 +,P25 - Problemas de las etapas de la vida en ad...,NaN,13


In [15]:
psico_prim.isna().sum()

Año                       0
CCAA                      0
Sexo                      0
Nivel_Renta               0
Edad                      0
Problema_de_Salud         0
Casos                182993
CodCA                     0
dtype: int64

In [16]:
psico_prim.Casos.fillna(0.,inplace = True)

In [17]:
psico_prim[["cod_salud","problema_salud"]]= psico_prim.Problema_de_Salud.str.split(" - ",expand=True)

In [18]:
psico_prim.drop('Problema_de_Salud',axis=1,inplace = True)

## equivalencias codigos diagnósticos con CEI10

In [19]:
trans_cod = {'P01': 'F40-49','P02': 'F40-49','P03': 'F40-49','P04': 'F40-49','P05':'F00-F09','P06': 'F30-39','P07': 'F30-39',
'P08': 'F30-39','P09': 'F30-39','P09-07': 'F30-39','P10': 'F90-F98','P11': 'F90-F98','P12': 'F90-F98','P13': 'F90-F98',
'P15': 'F10-F19','P16': 'F10-F19','P17': 'F10-F19','P18': 'F10-F19','P19': 'F10-F19','P20':'F00-F09','P22+23': 'F90-F98',
'P24': 'F80-89','P25': 'F60-69','P27': 'F99','P28': 'F70-79','P29': 'F99','P70':'F00-F09','P71': 'F20-29','P72': 'F20-29',
'P73': 'F20-29','P74': 'F40-49','P75': 'F40-49','P76': 'F30-39','P77': 'F30-39','P78': 'F40-49','P79': 'F40-49','P80': 'F60-69',
'P81': 'F90-F98','P82': 'F40-49','P82/02': 'F40-49','P85': 'F70-79','P86': 'F50-59','P98': 'F20-29','P99': 'F99',
}

In [20]:
psico_prim['cei10'] = psico_prim.cod_salud.map(trans_cod)

In [21]:
psico_prim.sample()

,Año,CCAA,Sexo,Nivel_Renta,Edad,Casos,CodCA,cod_salud,problema_salud,cei10
39956,2013,Madrid,Mujer,desconocida,15-34,0.0,13,P05,Sentimientos/conducta senil,F00-F09


## tipo columna casos

In [22]:
psico_prim.Casos =psico_prim.Casos.astype('int64')

In [23]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Edad,Casos,CodCA,cod_salud,problema_salud,cei10
67037,2014,Murcia,Mujer,desconocida,35-64,0,14,P29,Otros signos/síntomas psicológicos/mentales,F99
111695,2016,Cataluña,Mujer,+ 100000,00-14,0,09,P27,Miedo a una enfermedad mental,F99
143905,2017,Extremadura,Hombre,muy_baja,15-34,0,11,P29,Otros signos/síntomas psicológicos/mentales,F99
81854,2015,Castilla y León,Mujer,- 18000,65 +,0,07,P15,Abuso crónico del alcohol,F10-F19
189790,2019,Baleares,Hombre,+ 100000,35-64,0,04,P19,Abuso de drogas,F10-F19


In [24]:
psico_prim.to_csv('../data/psico_atprimaria.csv',index=False)